In [2]:
import scanpy as sc
import numpy as np
import functools
import jax
from cfp.metrics import compute_metrics, compute_mean_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

In [3]:
split = 4

In [4]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [5]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)


In [6]:
adata_ood.obs.head()

,sample,Size_Factor,n.umi,RT_well,Drug1,Drug2,Well,n_genes,n_genes_by_counts,total_counts,...,split,control,cell_type,cell_line,smiles_drug_1,smiles_drug_2,ood_1,ood_2,ood_3,ood_4
Cell,,,,,,,,,,,,,,,,,,,,,
A01_A02_RT_BC_13_Lig_BC_1,sciPlex_theis,1.083277,2908,RT_13,Cediranib,PCI-34051,B1,1896,1895,2907.0,...,ood,0,A549,A549,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,COC1=CC=C(C=C1)CN2C=CC3=C2C=C(C=C3)C(=O)NO,not ood,not ood,not ood,Cediranib+PCI-34051
A01_A02_RT_BC_13_Lig_BC_15,sciPlex_theis,0.946936,2542,RT_13,Cediranib,PCI-34051,B1,1655,1652,2539.0,...,ood,0,A549,A549,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,COC1=CC=C(C=C1)CN2C=CC3=C2C=C(C=C3)C(=O)NO,not ood,not ood,not ood,Cediranib+PCI-34051
A01_A02_RT_BC_13_Lig_BC_20,sciPlex_theis,1.765727,4740,RT_13,Cediranib,PCI-34051,B1,2823,2821,4738.0,...,ood,0,A549,A549,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,COC1=CC=C(C=C1)CN2C=CC3=C2C=C(C=C3)C(=O)NO,not ood,not ood,not ood,Cediranib+PCI-34051
A01_A02_RT_BC_13_Lig_BC_36,sciPlex_theis,1.394328,3743,RT_13,Cediranib,PCI-34051,B1,2298,2296,3741.0,...,ood,0,A549,A549,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,COC1=CC=C(C=C1)CN2C=CC3=C2C=C(C=C3)C(=O)NO,not ood,not ood,not ood,Cediranib+PCI-34051
A01_A02_RT_BC_13_Lig_BC_39,sciPlex_theis,0.885844,2378,RT_13,Cediranib,PCI-34051,B1,1638,1637,2377.0,...,ood,0,A549,A549,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,COC1=CC=C(C=C1)CN2C=CC3=C2C=C(C=C3)C(=O)NO,not ood,not ood,not ood,Cediranib+PCI-34051


In [7]:
adata_ood.obs["condition"].cat.categories

Index(['Cediranib+PCI-34051', 'Givinostat+Crizotinib', 'Givinostat+SRT2104',
       'Givinostat+Sorafenib', 'control', 'control+Givinostat',
       'control+SRT2104'],
      dtype='object')

In [8]:
adata_train.obs["condition"].cat.categories

Index(['Alvespimycin+Pirarubicin', 'Dacinostat+Danusertib',
       'Dacinostat+Dasatinib', 'Dacinostat+PCI-34051', 'Givinostat+Carmofur',
       'Givinostat+Cediranib', 'Givinostat+Curcumin', 'Givinostat+Dasatinib',
       'Givinostat+SRT1720', 'Givinostat+Tanespimycin',
       'Panobinostat+Alvespimycin', 'Panobinostat+Crizotinib',
       'Panobinostat+Curcumin', 'Panobinostat+Dasatinib',
       'Panobinostat+PCI-34051', 'Panobinostat+SRT1720',
       'Panobinostat+SRT2104', 'Panobinostat+SRT3025',
       'Panobinostat+Sorafenib', 'SRT2104+Alvespimycin', 'SRT3025+Cediranib',
       'control', 'control+Alvespimycin', 'control+Dacinostat',
       'control+Dasatinib', 'control+Panobinostat'],
      dtype='object')

In [9]:
['Alvespimycin+Pirarubicin', 'Dacinostat+Danusertib',
       'Dacinostat+Dasatinib', 'Dacinostat+PCI-34051', 'Givinostat+Carmofur',
       'Givinostat+Cediranib', 'Givinostat+Curcumin', 'Givinostat+Dasatinib',
       'Givinostat+SRT1720', 'Givinostat+Tanespimycin',
       'Panobinostat+Alvespimycin', 'Panobinostat+Crizotinib',
       'Panobinostat+Curcumin', 'Panobinostat+Dasatinib',
       'Panobinostat+PCI-34051', 'Panobinostat+SRT1720',
       'Panobinostat+SRT2104', 'Panobinostat+SRT3025',
       'Panobinostat+Sorafenib', 'SRT2104+Alvespimycin', 'SRT3025+Cediranib',
       'control', 'control+Alvespimycin', 'control+Dacinostat',
       'control+Dasatinib', 'control+Panobinostat']

['Alvespimycin+Pirarubicin',
 'Dacinostat+Danusertib',
 'Dacinostat+Dasatinib',
 'Dacinostat+PCI-34051',
 'Givinostat+Carmofur',
 'Givinostat+Cediranib',
 'Givinostat+Curcumin',
 'Givinostat+Dasatinib',
 'Givinostat+SRT1720',
 'Givinostat+Tanespimycin',
 'Panobinostat+Alvespimycin',
 'Panobinostat+Crizotinib',
 'Panobinostat+Curcumin',
 'Panobinostat+Dasatinib',
 'Panobinostat+PCI-34051',
 'Panobinostat+SRT1720',
 'Panobinostat+SRT2104',
 'Panobinostat+SRT3025',
 'Panobinostat+Sorafenib',
 'SRT2104+Alvespimycin',
 'SRT3025+Cediranib',
 'control',
 'control+Alvespimycin',
 'control+Dacinostat',
 'control+Dasatinib',
 'control+Panobinostat']

In [10]:
control = adata_ood[adata_ood.obs["condition"]=="control"].X.toarray()
rng = np.random.default_rng(0)
drug_effect = {}
for cond in adata_train.obs["condition"].cat.categories:
    drug_effect[cond] = rng.choice(adata_train[adata_train.obs["condition"]==cond].X.toarray(), 500) - control

drug_effect["control"] = control # note this is not a displacement vector, but the control vector
    

In [11]:
drug_effect_ood = {}

drug_effect_ood['Cediranib+PCI-34051'] = (
    drug_effect['control'] 
    + drug_effect['Dacinostat+PCI-34051'] 
    - drug_effect['control+Dacinostat']
    + drug_effect['Givinostat+Cediranib']
    - drug_effect['Givinostat+Dasatinib']
    + drug_effect['control+Dasatinib']
)


drug_effect_ood['Givinostat+Crizotinib'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+Dasatinib'] 
    - drug_effect['control+Dasatinib']
    + drug_effect['Panobinostat+Crizotinib']
    - drug_effect['control+Panobinostat']
)

drug_effect_ood['Givinostat+SRT2104'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+Dasatinib'] 
    - drug_effect['control+Dasatinib']
    + drug_effect['Panobinostat+SRT2104']
    - drug_effect['control+Panobinostat']
)

drug_effect_ood['Givinostat+Sorafenib'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+Dasatinib'] 
    - drug_effect['control+Dasatinib']
    + drug_effect['Panobinostat+Sorafenib']
    - drug_effect['control+Panobinostat']
)

drug_effect_ood['control+Givinostat'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+Dasatinib'] 
    - drug_effect['control+Dasatinib']
)


drug_effect_ood['control+SRT2104'] = (
    drug_effect['control'] 
    + drug_effect['Panobinostat+SRT2104'] 
    - drug_effect['control+Panobinostat']
)


In [12]:
adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [13]:
import anndata as ad
import pandas as pd
all_data = []
conditions = []

for condition, array in drug_effect_ood.items():
    all_data.append(array)
    conditions.extend([condition] * array.shape[0])

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)

# Create a DataFrame for the .obs attribute
obs_data = pd.DataFrame({
    'condition': conditions
})

# Create the Anndata object
adata_pred_ood = ad.AnnData(X=all_data_array, obs=obs_data)

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [14]:
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood)
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].X
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

In [15]:

ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [16]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)


In [17]:
adata_pred_ood

AnnData object with n_obs × n_vars = 3000 × 2000
    obs: 'condition'
    obsm: 'X_pca'

In [18]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")


In [19]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.8495323260625204,
 'deg_ood_sinkhorn_div_1': 36.72664753595988,
 'deg_ood_sinkhorn_div_10': 17.33673032124837,
 'deg_ood_sinkhorn_div_100': 4.931852658589681,
 'deg_ood_e_distance': 8.879143970137717,
 'deg_ood_mmd': 0.04949475700656573}

In [20]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [21]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.8678234020868937,
 'decoded_ood_sinkhorn_div_1': 428.4222717285156,
 'decoded_ood_sinkhorn_div_10': 396.74501037597656,
 'decoded_ood_sinkhorn_div_100': 112.9091796875,
 'decoded_ood_e_distance': 23.468859365447713,
 'decoded_ood_mmd': 0.07495708515246709}

In [22]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': -3.5613041818141937,
 'encoded_ood_sinkhorn_div_1': 22.012898604075115,
 'encoded_ood_sinkhorn_div_10': 14.328720728556315,
 'encoded_ood_sinkhorn_div_100': 8.085860252380371,
 'encoded_ood_e_distance': 14.226094096717285,
 'encoded_ood_mmd': 0.05186058821467062}

In [23]:
ood_metrics_encoded

{'Cediranib+PCI-34051': {'r_squared': -1.350844383239746,
  'sinkhorn_div_1': 27.84992790222168,
  'sinkhorn_div_10': 19.07388687133789,
  'sinkhorn_div_100': 8.750728607177734,
  'e_distance': 14.268831515512446,
  'mmd': 0.059177533},
 'Givinostat+Crizotinib': {'r_squared': -7.093904495239258,
  'sinkhorn_div_1': 20.947816848754883,
  'sinkhorn_div_10': 12.781055450439453,
  'sinkhorn_div_100': 6.50299072265625,
  'e_distance': 11.095312303716142,
  'mmd': 0.0460455},
 'Givinostat+SRT2104': {'r_squared': -4.896747589111328,
  'sinkhorn_div_1': 19.250782012939453,
  'sinkhorn_div_10': 11.047882080078125,
  'sinkhorn_div_100': 4.548213958740234,
  'e_distance': 7.11048342932682,
  'mmd': 0.03969086},
 'Givinostat+Sorafenib': {'r_squared': -3.5398483276367188,
  'sinkhorn_div_1': 42.84703826904297,
  'sinkhorn_div_10': 33.30573272705078,
  'sinkhorn_div_100': 23.891921997070312,
  'e_distance': 44.71745610556626,
  'mmd': 0.12447248},
 'control+Givinostat': {'r_squared': -5.389335155487

In [24]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/additive_model"

In [25]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
